# Libraries

In [1]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [2]:
# Get the current user's name
user = getuser()

# Import the functions from the functions notebook
%run functions_uefa.ipynb


In [3]:


# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\wc_goals_men.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,year,stage,score,time,home_team,away_team,stadium_name,stadium_city,stadium_attendance,referee_name,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,1991,Group A,4–0,20:45,China,Norway,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,...,22,0,4,0,0,0,0,0,1991-11-16,16 November 1991
1,1991,Group A,4–0,20:45,China,Norway,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,...,45,0,4,0,0,0,0,0,1991-11-16,16 November 1991
2,1991,Group A,4–0,20:45,China,Norway,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,...,50,0,4,0,0,0,0,0,1991-11-16,16 November 1991
3,1991,Group A,4–0,20:45,China,Norway,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,...,75,0,4,0,0,0,0,0,1991-11-16,16 November 1991
4,1991,Group A,3–0,19:45,Denmark,New Zealand,Tianhe Stadium,Guangzhou,140008,Omer Yengo,...,15,0,3,0,0,0,0,0,1991-11-17,17 November 1991


# Clean, transfrom, create variables

## time

In [4]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [5]:
# Convert 'long_date' column to datetime format
df['long_date'] = pd.to_datetime(df['long_date'], format='%d %B %Y', errors='coerce')

# Extract the year from 'long_date' and create a new column 'year'
df['year'] = df['long_date'].dt.year


# Extract relevant columns

In [6]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

# Remove duplicates
goals_df = goals_df.drop_duplicates()

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
0,1991,Group A,China,Norway,China,22,1991-11-16,20:45,4–0
1,1991,Group A,China,Norway,China,45,1991-11-16,20:45,4–0
2,1991,Group A,China,Norway,China,50,1991-11-16,20:45,4–0
3,1991,Group A,China,Norway,China,75,1991-11-16,20:45,4–0
4,1991,Group A,Denmark,New Zealand,Denmark,15,1991-11-17,19:45,3–0


# Recreate Leauge Table after first two matchdays

In [7]:

# Assuming goals_df is your original dataset
agg_goals_before_last_day, goals_last_day_sorted = process_goals_data(goals_df)


In [8]:
home_games, away_games = aggregate_home_away_points(agg_goals_before_last_day)

In [9]:
# Filter for year == 2021 and stage == 'Group F'
home_f_2021 = home_games[(home_games['year'] == 2021) & (home_games['stage'] == 'Group F')]
home_f_2021

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home


In [10]:
# Filter for year == 2021 and stage == 'Group F'
away_f_2021 = away_games[(away_games['year'] == 2021) & (away_games['stage'] == 'Group F')]
away_f_2021

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away


## aggregate data after first two matches following 

### UEFA criteria (first h2h, then aggregate)

In [11]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)

Tie in 1991, Stage: Group A, between China and Denmark (Points: 3.0)
Tie resolved in favor of China
Tie in 1991, Stage: Group B, between Brazil and Sweden (Points: 2.0)
Tie resolved in favor of Sweden
Tie in 1991, Stage: Group C, between Germany and Italy (Points: 4.0)
Tie resolved in favor of Germany
Tie in 1991, Stage: Group C, between Chinese Taipei and Nigeria (Points: 0.0)
Tie resolved in favor of Nigeria
Tie in 1995, Stage: Group A, between Brazil and Germany (Points: 3.0)
Tie resolved in favor of Germany
Tie in 1995, Stage: Group A, between Germany and Sweden (Points: 3.0)
Tie resolved in favor of Germany
Tie in 1995, Stage: Group A, between Sweden and Japan (Points: 3.0)
Tie resolved in favor of Sweden
Tie in 1995, Stage: Group B, between Nigeria and Canada (Points: 1.0)
Tie resolved in favor of Canada
Tie in 1995, Stage: Group C, between China and United States (Points: 4.0)
Tie resolved in favor of China
Tie in 2007, Stage: Group B, between North Korea and United States (Poin

In [12]:
# Filtering all_games_before_last for the year 2021 and stage 'Group F'
groupf2021 = all_games_before_last[
    (all_games_before_last['year'] == 2022) & 
    (all_games_before_last['stage'] == 'Group E')
]

display(groupf2021)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,tie_won,standing
136,2022,Group E,Spain,8.0,1.0,4.0,7.0,2.0,no need,0,1
137,2022,Group E,Costa Rica,1.0,7.0,3.0,-6.0,2.0,Costa Rica,1,2
138,2022,Group E,Japan,2.0,2.0,3.0,0.0,2.0,Costa Rica,0,3
139,2022,Group E,Germany,2.0,3.0,1.0,-1.0,2.0,no need,0,4


# Recreate league table after last match day

### uefa criteria 

In [13]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = uefa_final_euro(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[[ 'year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 1991, Group A Before Last Match Goals ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
      China           4.0                 6.0                   2.0                    4.0                      3.0                          1
    Denmark           4.0                 5.0                   2.0                    3.0                      3.0                          2
     Norway           3.0                 4.0                   4.0                    0.0                      2.0                          3
New Zealand           1.0                 0.0                   7.0                   -7.0                      0.0                          4


Analyzing goal: 14 minute, Player team: Norway, Home: Norway, Away: Denmark

=== Updated Standings After This Goal ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_differen

In [14]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\tb_wc_uefa_men.xlsx'
changes_df_euro.to_excel(file_path, index=False)


# group composition tracking

In [15]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Display the final DataFrame
display(final_composition_changes_df)




=== Initial Standings for Group A, 1991 (with 0-0 points added) ===
       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
      China           4.0                 6.0                   2.0                    4.0                          1
    Denmark           4.0                 5.0                   2.0                    3.0                          2
     Norway           3.0                 4.0                   4.0                    0.0                          3
New Zealand           1.0                 0.0                   7.0                   -7.0                          4



=== Standings after goal at minute 14 in Group A, edition 1991 ===
       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_standing  tied_won
     Norway           5.0                 5.0                   4.0                    1.0                   1         0
      China           4.0    

,year,stage,change_num,goal_time,home_team,away_team,scorer_team,new_top_teams,1st,2nd,3rd,scorer_nationality
0,1991,Group A,0,initial,None,None,None,"[Denmark, Norway, China]",China,Denmark,Norway,NaN
1,1991,Group B,0,initial,None,None,None,"[United States, Brazil, Sweden]",United States,Sweden,Brazil,NaN
2,1991,Group C,0,initial,None,None,None,"[Germany, Italy, Nigeria]",Germany,Italy,Nigeria,NaN
3,1991,Group C,1,38,Chinese Taipei,Nigeria,None,"[Germany, Italy, Chinese Taipei]",Germany,Italy,Chinese Taipei,Chinese Taipei
4,1995,Group A,0,initial,None,None,None,"[Germany, Sweden]",Germany,Sweden,Brazil,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
76,2023,Group E,0,initial,None,None,None,"[United States, Portugal]",United States,Portugal,Vietnam,NaN
77,2023,Group F,0,initial,None,None,None,"[France, Brazil]",Brazil,France,Panama,NaN
78,2023,Group G,0,initial,None,None,None,"[Argentina, Sweden]",Sweden,Argentina,South Africa,NaN
79,2023,Group G,1,32,South Africa,Italy,None,"[Sweden, South Africa]",Sweden,South Africa,Argentina,South Africa


In [16]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\standings_wc_uefa_men.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)


# best four third placed

In [17]:
# Call the function to get the DataFrame
final_df = best_four_third_placed_wc(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day)

# Define the file path and save to Excel
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\third_teams_wc_men.xlsx'
final_df.to_excel(file_path, index=False)



--- Processing Year: (1991,) ---

=== Initial Standings for Year (1991,) Before Processing Any Goals ===

          team   stage  before_last_game_points  before_last_game_goals_scored  before_last_game_goals_conceded  total_goal_difference
         China Group A                      3.0                            6.0                              2.0                    4.0
       Denmark Group A                      3.0                            5.0                              2.0                    3.0
        Norway Group A                      2.0                            4.0                              4.0                    0.0
   New Zealand Group A                      0.0                            0.0                              7.0                   -7.0
 United States Group B                      4.0                            8.0                              2.0                    6.0
        Sweden Group B                      2.0                           10.0     

In [18]:
# Count the number of observations where the year is 1986
count_1986 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1986].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2016: {count_1986}")

# Count the number of observations where the year is 1990
count_1990 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1990].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2020: {count_1990}")

# Count the number of observations where the year is 1994
count_1994 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1994].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2024: {count_1994}")


Number of observations in goals_last_day_sorted for the year 2016: 0
Number of observations in goals_last_day_sorted for the year 2020: 0
Number of observations in goals_last_day_sorted for the year 2024: 0
